In [247]:
import requests
import json
import pandas as pd

In [248]:
# API endpoint from the newly deployed service

API_URL = "https://zfgp45ih7i.execute-api.eu-west-1.amazonaws.com/sandbox/api/search"
API_KEY = "XYZ38746G38B7RB46GBER"

headers = {
    "Content-Type": "application/json",
    "x-api-key": API_KEY
}

# Edit the below to get different data
payload = {
  "query_text": "Welsh politics",
  "result_size": 50,
  "include_highlights":True,
  "include_smart_tags":True,
  "timerange": "7d"
}

response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
json_response = response.json()

#json_response

In [249]:
# ['1d', '7d', '30d', '180d', '365d', 'all']

In [250]:
import pandas as pd

#Create dataframe
df = pd.DataFrame(json_response['results'])

#Reformat date
df['published_date'] = pd.to_datetime(df['timestamp'], errors='coerce').dt.date


In [251]:
#iterating with iterrows()

for index, row in df.iterrows():
  print(row['title'])

Welsh Government warned disability plan lacks key targets
Planet Expands Business with Welsh Government for Land and Natural Resource Management
Planet Expands Business with Welsh Government for Land and Natural Resource Management | PL Stock News
First Minister and Future Generations Commissioner visit Borth climate change project
£500,000 to 'significantly upgrade' public toilets on key travel routes across Powys
Strata Florida visitor centre reopens thanks to local support
More than 80 Powys patients left waiting two years for treatment, says Wales First Minister
Welsh Conservatives seek farmland solar panel ban
Dental care row continues as politician weighs in on local NHS crisis
Britain's water sector 'failing' and needs stronger regulation, damning review finds
Wales Offered Stark Choice Between ‘Reform-Led Government or Plaid Cymru’ – as Labour’s 26-year Rule Starts to Crumble
Tories demand farm solar panel ban to protect Welsh food security
Community heroes celebrated during Vo

In [252]:
!pip install nltk
import nltk
nltk.download('punkt')         # For word tokenization
nltk.download('averaged_perceptron_tagger') # For Part-of-Speech tagging
nltk.download('maxent_ne_chunker') # For named entity chunking
nltk.download('words')         # Used by ne_chunker
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker_tab is already up-to-date!


True

In [253]:
#get NER with nltk

#tokenize the text
from nltk.tokenize import word_tokenize
def tokenize(text):
  tokens = word_tokenize(text)
  return tokens

#tag parts of speech
from nltk.tag import pos_tag

def POSTag(tokens):
  pos_tags = pos_tag(tokens)
  return pos_tags

#named entity recognition
from nltk.chunk import ne_chunk

def NER(pos_tags):
  ner_tree = ne_chunk(pos_tags)
  return ner_tree



In [254]:
#trees = []
#
#for index, row in df.iterrows():
#  trees.append(NER(POSTag(tokenize(row['summary']))))

In [255]:
#def get_named_entities(tree):
#  named_entities = []
#  for chunk in tree:
#      if hasattr(chunk, 'label'):
#          entity = " ".join(c[0] for c in chunk)
#          entity_type = chunk.label()
#          named_entities.append((entity, entity_type))
#  return named_entities

In [256]:
#NEs = [get_named_entities(tree) for tree in trees]

In [257]:
#These are a bit hit and miss
#NEs

[[('Welsh Government', 'PERSON'),
  ('Mark Isherwood', 'PERSON'),
  ('Senedd', 'ORGANIZATION')],
 [('Planet', 'PERSON'),
  ('Labs PBC', 'PERSON'),
  ('NYSE', 'ORGANIZATION'),
  ('Planet', 'PERSON'),
  ('Welsh', 'GPE'),
  ('Planet', 'PERSON'),
  ('SkySat', 'ORGANIZATION'),
  ('PlanetScope', 'ORGANIZATION'),
  ('Planet Fusion', 'PERSON')],
 [('Planet', 'PERSON'),
  ('Labs PBC', 'PERSON'),
  ('Planet', 'PERSON'),
  ('SkySat', 'ORGANIZATION'),
  ('PlanetScope', 'ORGANIZATION'),
  ('Campus', 'GPE')],
 [('Eluned Morgan', 'PERSON'),
  ('Commissioner Derek Walker', 'ORGANIZATION'),
  ('Welsh', 'ORGANIZATION'),
  ('Energy Service', 'PERSON')],
 [('Welsh', 'GPE'),
  ('Powys', 'GPE'),
  ('Llandrindod Wells', 'PERSON'),
  ('Mid Wales', 'GPE')],
 [('Cadw', 'GPE'),
  ('Welsh', 'ORGANIZATION'),
  ('Strata Florida Trust', 'ORGANIZATION'),
  ('StratA Florida', 'ORGANIZATION')],
 [('Powys', 'GPE'), ('Montgomeryshire MS Russell George', 'PERSON')],
 [('Senedd', 'ORGANIZATION'),
  ('Welsh', 'GPE'),
  ('Wa

In [258]:
# NER with Spacy
#!python -m spacy download en_core_web_md

In [259]:
import spacy
nlp = spacy.load("en_core_web_md")

In [260]:
def get_NEs_Spacy(text):
  NEs = []
  doc = nlp(text)
  for ent in doc.ents:
    NEs.append((ent.text, ent.label_))
  return NEs

In [261]:
spacy_nes = [get_NEs_Spacy(s) for s in df['summary']]

In [262]:
#spacy_nes

#Add to dataframe
df['named_entities'] = spacy_nes

In [263]:
#Create columns of people and places
people = []
places = []

for index, row in df.iterrows():
  person = []
  gpe = []
  for entity in row['named_entities']:
    if entity[1] == "PERSON":
      person.append(entity[0])
    if entity[1] == "GPE":
      gpe.append(entity[0])
  people.append(person)
  places.append(gpe)

#Add to dataframe
df['people'] = people
df['places'] = places


In [ ]:
#emotion detection with huggingface

from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="j-hartmann/emotion-english-distilroberta-base")

emotion = []
for s in df['summary']:
  result = classifier(s, top_k=3)
  emotion.append(result)

#Add to dataframe
df['emotions'] = emotion

#df.groupby('published_date')

Device set to use cpu


In [265]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [266]:
## Clean data
import re
def clean_text(text):
    """
    - Convert to lowercase
    - Remove URLs
    - Remove punctuation / non-alpha
    - Collapse multiple spaces
    """
    if not isinstance(text, str):
        return ""
    # Remove URLs (very basic)
    text = re.sub(r"http\S+|www\.\S+", "", text)
    # Lowercase
    text = text.lower()
    # Keep only letters and spaces
    text = re.sub(r"[^a-z\s]", " ", text)
    # Collapse multiple spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

df['clean_summary'] = df['summary'].apply(clean_text)


In [267]:
## Sentiment analysis example

sia = SentimentIntensityAnalyzer()

def get_sentiment_scores(text):
    """
    Returns a dict with these keys:
       - neg: negative sentiment score
       - neu: neutral score
       - pos: positive score
       - compound: normalized, weighted composite (-1 to +1)
    """
    return sia.polarity_scores(text)

# Apply to each summary
df['sentiment'] = df['clean_summary'].apply(get_sentiment_scores)

# Split into separate columns if you like
df['sent_neg'] = df['sentiment'].apply(lambda d: d['neg'])
df['sent_neu'] = df['sentiment'].apply(lambda d: d['neu'])
df['sent_pos'] = df['sentiment'].apply(lambda d: d['pos'])
df['sent_compound'] = df['sentiment'].apply(lambda d: d['compound'])

# Quick look at top 5 compound scores
print(df[['clean_summary', 'sent_compound']].sort_values(by='sent_compound', ascending=False).head())
print(df[['clean_summary', 'sent_compound']].sort_values(by='sent_compound').head())


                                        clean_summary  sent_compound
5   cadw the welsh government s historic environme...         0.9587
2   planet labs pbc pl is a premier provider of gl...         0.9169
16  welsh government grant awarded to powys county...         0.9118
23  national assembly for wales now welsh governme...         0.9118
29  brazil updates list of trade prohibitions foll...         0.8591
                                        clean_summary  sent_compound
47  blackweir live will see world famous artists l...        -0.8847
26  royal welsh show to ban livestock from england...        -0.8176
20  a heads of the valleys in southern wales has b...        -0.7351
6   first minister stated that there are about peo...        -0.7096
39  about items are missing at the national librar...        -0.6486


In [273]:
new_additions = df[['timestamp', 'title', 'published_date', 'people', 'places', 'emotions']]
print(new_additions.sort_values(by='timestamp'))

                    timestamp  \
17        2025-06-03T09:13:13   
18  2025-06-03T10:41:54+00:00   
9   2025-06-03T11:44:00+00:00   
41  2025-06-03T12:48:08+00:00   
2   2025-06-03T13:42:24+00:00   
24  2025-06-03T14:13:30+00:00   
1   2025-06-03T14:14:28+00:00   
45  2025-06-04T07:53:07+00:00   
33  2025-06-04T08:09:00+00:00   
21  2025-06-04T09:46:23+00:00   
6         2025-06-04T10:30:00   
10  2025-06-04T15:54:15+00:00   
29  2025-06-04T19:54:12+00:00   
49  2025-06-04T20:12:54+00:00   
27  2025-06-05T07:33:27+00:00   
44  2025-06-05T07:51:24+00:00   
22  2025-06-05T08:15:06+00:00   
42  2025-06-05T22:01:04+00:00   
39  2025-06-05T22:01:04+00:00   
0   2025-06-06T05:16:25+00:00   
4   2025-06-06T12:00:00+00:00   
14  2025-06-06T12:35:48+00:00   
37  2025-06-06T15:32:42+00:00   
32  2025-06-07T08:51:16+00:00   
36        2025-06-07T09:30:27   
7   2025-06-07T11:02:49+00:00   
19  2025-06-07T12:40:03+00:00   
48        2025-06-07T17:00:24   
23                 2025-06-08   
11        